# 第 4 章　文本和字节序列

Python 3 明确区分了人类可读的文本字符串和原始的字节序列。

本章将要讨论 Unicode 字符串、二进制序列，以及在二者之间转换时使用的编码。

## Byte Objects vs String

* Byte objects are sequence of **Bytes**, whereas Strings are sequence of **characters**.
* Byte objects are in **machine readable** form internally, Strings are only in **human readable** form.
* Since Byte objects are machine readable, they can be **directly stored on the disk**. Whereas, Strings **need encoding** before which they can be stored on disk.

## 4.1 字符问题

显然，一个字符串是一个字符序列。以下讨论字符。

* 从 Python 3 的str 对象中获取的元素是 Unicode 字符

* 从 Python 2 的 str 对象中获取的是原始字节序列

Unicode 标准把字符的标识和具体的字节表述进行了如下的明确区分。

* 字符的标识，即码位，是 `0~1 114 111`的数字（十进制），在Unicode 标准中以 4~6 个十六进制数字表示，而且加前缀“U+”。例
如，字母 A 的码位是 `U+0041`，欧元符号的码位是 `U+20AC`，高音谱号的码位是 `U+1D11E`。
在 Unicode 6.3 中（这是 Python 3.4 使用的标准），约 10% 的有效码位有对应的字符。

* 字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换时使用的算法。
在 UTF-8 编码中， A（U+0041）的码位编码成单个字节 `\x41`，而在 UTF-16LE 编码中编码成两个字节`\x41\x00`。
再举个例子，欧元符号（U+20AC）在 UTF-8 编码中是三个字节——`\xe2\x82\xac`，而在 UTF-16LE 中编码成两个字节： `\xac\x20`

把码位转换成字节序列的过程是**编码**；把字节序列转换成码位的过程是**解码**。

>字符串是文本的抽象表示。字符串由字符组成，字符则是与任何特定二进制表示无关的抽象实体。
>在操作字符串时，我们生活在幸福的无知之中。我们可以对字符串进行分割和分片，可以拼接和搜索字符串。
>我们并不关心它们内部是怎么表示的，字符串里的每个字符要用几个字节保存。
>只有在将字符串编码成字节包（例如，为了在信道上发送它们）或从字节包解码字符串（反向操作）时，我们才会开始关注这点。

![str&bytes](../image/Python_str_&_bytes.png)

In [1]:
# 字符串有 4 个 Unicode 字符
s = 'café'
len(s)

4

In [2]:
#  str 对象编码成 bytes 对象
b = s.encode('utf8')
len(b)

5

In [3]:
b

b'caf\xc3\xa9'

In [4]:
b.decode('utf8')

'café'

Python3 的`str`是字符串序列，`bytes`是字节序列，打印的时候前面有`b`

## 4.2　字节概要

Python 2.6 添加的可变 `bytearray` 类型。

Python 3 引入的不可变`bytes` 类型

bytes 或 bytearray 对象的各个元素是介于 0~255（含）之间的整数

In [5]:
# bytes 对象可以从 str 对象使用给定的编码
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [6]:
cafe[0]

99

In [7]:
# bytes 对象的切片还是 bytes 对象，即使是只有一个字节的切片。
cafe[:1]

b'c'

In [8]:
#  bytearray 对象没有字面量句法，而是以 bytearray() 和字节序列字面量参数的形式显示
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [9]:
# bytearray 对象的切片还是 bytearray 对象
cafe_arr[-1:]

bytearray(b'\xa9')

虽然二进制序列其实是整数序列，但是它们的字面量表示法表明其中有ASCII 文本

因此，各个字节的值可能会使用下列三种不同的方式显示。

* 可打印的 ASCII 范围内的字节（从空格到 ~），使用 ASCII 字符本身。

* 制表符、换行符、回车符和 \ 对应的字节，使用转义序列\t、 \n、 \r 和 \\。

* 其他字节的值，使用十六进制转义序列（例如， \x00 是空字节）。


除了格式化方法（`format` 和 `format_map`）和几个处理 Unicode 数据的方法（包括`casefold`、 `isdecimal`、 `isidentifier`、 `isnumeric`、 `isprintable`
和 `encode`）之外， `str` 类型的其他方法都支持 `bytes` 和 `bytearray`
类型。

我们可以使用熟悉的字符串方法处理二进制序列，如`endswith`、 `replace`、 `strip`、 `translate`、 `upper` 等，
只有少数几个其他方法的参数是 `bytes` 对象，而不是 `str` 对象。

二进制序列有个类方法是 str 没有的，名为 `fromhex`，它的作用是解析十六进制数字对（数字对之间的空格是可选的），构建二进制序列

## 4.3　基本的编解码器

Python 自带了超过 100 种编解码器（`codec`, `encoder/decoder`），用于在文本和字节之间相互转换。

每个编解码器都有一个名称，如'utf_8'，而且经常有几个别名，如 'utf8'、 'utf-8' 和 'U8'。

这些名称可以传给 `open()`、 `str.encode()`、 `bytes.decode()` 等函数的 encoding 参数。

一些典型编码，介绍如下

* latin1（即 iso8859_1）
    一种重要的编码，是其他编码的基础，例如 cp1252 和
Unicode（注意， latin1 与 cp1252 的字节值是一样的，甚至连码位
也相同）。

* cp1252
    Microsoft 制定的 latin1 超集，添加了有用的符号，例如弯引号
和€（欧元）；有些 Windows 应用把它称为“ANSI”，但它并不是 ANSI
标准。

* cp437
    IBM PC 最初的字符集，包含框图符号。与后来出现的 latin1 不
兼容。

* gb2312
　　用于编码简体中文的陈旧标准；这是亚洲语言中使用较广泛的多字
节编码之一。

* utf-8
　　目前 Web 中最常见的 8 位编码； 与 ASCII 兼容（纯 ASCII 文本是
有效的 UTF-8 文本）。





## 4.4　了解编解码问题

### 4.4.1　处理UnicodeEncodeError

多数非 UTF 编解码器只能处理 Unicode 字符的一小部分子集。把文本转换成字节序列时，
如果目标编码中没有定义某个字符，那就会抛出`UnicodeEncodeError` 异常，
除非把 errors 参数传给编码方法或函数，对错误进行特殊处理。

### 4.4.2　处理UnicodeDecodeError

不是每一个字节都包含有效的 ASCII 字符，也不是每一个字符序列都是有效的 UTF-8 或 UTF-16。
因此，把二进制序列转换成文本时，如果假设是这两个编码中的一个，遇到无法转换的字节序列时会抛出
`UnicodeDecodeError`。

很多陈旧的 8 位编码——如 'cp1252'、 'iso8859_1' 和'koi8_r'——能解码任何字节序列流而不抛出错误，例如随机噪声。
因此，如果程序使用错误的 8 位编码，解码过程悄无声息，而得到的是无用输出。

### 4.4.3　使用预期之外的编码加载模块时抛出的SyntaxError

Python 3 默认使用 UTF-8 编码源码， Python 2（从 2.5 开始）则默认使用
ASCII。

如果加载的 .py 模块中包含 UTF-8 之外的数据，而且没有声明编码，会得到类似下面的消息

>SyntaxError: Non-UTF-8 code starting with '\xe1' in file ola.py on line
1, but no encoding declared; see http://python.org/dev/peps/pep-0263/
for details
>

GNU/Linux 和 OS X 系统大都使用 UTF-8，因此打开在 Windows 系统中使用 cp1252 编码的 .py 文件时可能发生这种情况。

这个错误在Windows 版 Python 中也可能会发生，因为 Python 3 为所有平台设置的默认编码都是 UTF-8。

### 4.4.4　如何找出字节序列的编码

如何找出字节序列的编码？简单来说，不能。必须有人告诉你。

然而，就像人类语言也有规则和限制一样，只要假定字节流是人类可读的纯文本，就可能通过试探和分析找出编码。

二进制序列编码文本通常不会明确指明自己的编码，但是 UTF 格式可以在文本内容的开头添加一个字节序标记。

### 4.4.5 BOM：有用的鬼符

UTF-16 编码的序列开头有几个额外的字节，如下所示：


In [10]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

`b'\xff\xfe'`是 BOM，即字节序标记（ byte-ordermark），指明编码时使用 Intel CPU 的小字节序。

在小字节序设备中，各个码位的最低有效字节在前面：字母 'E' 的码位是 U+0045（十进制数 69），在字节偏移的第 2 位和第 3 位编码为 69
和 0。

在大字节序 CPU 中，编码顺序是相反的； 'E' 编码为 0 和 69。

为了避免混淆， UTF-16 编码在要编码的文本前面加上特殊的不可见字符 ZERO WIDTH NO-BREAK SPACE（ U+FEFF）。
在小字节序系统中，这个字符编码为 b'\xff\xfe'（十进制数 255, 254）。
因为按照设计， U+FFFE 字符不存在，在小字节序编码中，字节序列b'\xff\xfe' 必定是 ZERO WIDTH NO-BREAK SPACE，所以编解码器知道该用哪个字节序

UTF-8 的一大优势是，不管设备使用哪种字节序，生成的字节序列始终一致，因此不需要 BOM。
尽管如此，某些Windows 应用（尤其是 Notepad）依然会在 UTF-8 编码的文件中添加BOM；
而且， Excel 会根据有没有 BOM 确定文件是不是 UTF-8 编码，否则，它假设内容使用 Windows 代码页（codepage）编码。
 UTF-8 编码的 U+FEFF 字符是一个三字节序列： b'\xef\xbb\xbf'。
因此，如果文件以这三个字节开头，有可能是带有 BOM 的 UTF-8 文件。
然而，Python 不会因为文件以 b'\xef\xbb\xbf' 开头就自动假定它是 UTF-8 编码的

## 4.5　处理文本文件

处理文本的最佳实践是“Unicode 三明治”,要尽早把输入（例如读取文件时）的字节序列解码成字符串。

对输出来说，则要尽量晚地把字符串编码成字节序列。

例如，在 Django 中，视图应该输出 Unicode 字符串；Django 会负责把响应编码成字节序列，而且默认使用 UTF-8 编码。

Python 3 内置的 `open`函数会在读取文件时做必要的解码，以文本模式写入文件时还会做必要的编码，
所以调用 `my_file.read()` 方法得到的以及传给`my_file.write(text)` 方法的都是字符串对象。

## 4.6 为了正确比较而规范化Unicode字符串

## 4.7 Unicode文本排序
